# **Homework**: Data talks club data engineering zoomcamp Data loading workshop

Hello folks, let's practice what we learned - Loading data with the best practices of data engineering.

Here are the exercises we will do




In [ ]:
# Imports 
import dlt
import duckdb

# 1. Use a generator

Remember the concept of generator? Let's practice using them to futher our understanding of how they work.

Let's define a generator and then run it as practice.

**Answer the following questions:**

- **Question 1: What is the sum of the outputs of the generator for limit = 5?**
- **Question 2: What is the 13th number yielded**

I suggest practicing these questions without GPT as the purpose is to further your learning.

In [ ]:
def square_root_generator(limit):
    n = 1
    while n <= limit:
        yield n ** 0.5
        n += 1

# Example usage:
limit = 5
generator = square_root_generator(limit)

for sqrt_value in generator:
    print(sqrt_value)


In [ ]:
# Question 1
sum(square_root_generator(5))

In [ ]:
# Question 2
for i, value in enumerate(square_root_generator(13)):
    if i == 12: # 13th value has index = 12 (starts with 0)
        print(value)

# 2. Append a generator to a table with existing data


Below you have 2 generators. You will be tasked to load them to duckdb and answer some questions from the data

1. Load the first generator and calculate the sum of ages of all people. Make sure to only load it once.
2. Append the second generator to the same table as the first.
3. **After correctly appending the data, calculate the sum of all ages of people.**




In [ ]:
def people_1():
    for i in range(1, 6):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 25 + i, "City": "City_A"}

for person in people_1():
    print(person)


def people_2():
    for i in range(3, 9):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 30 + i, "City": "City_B", "Occupation": f"Job_{i}"}


for person in people_2():
    print(person)


In [ ]:
# define the connection to load to
pipeline = dlt.pipeline(destination='duckdb', dataset_name='people')

In [ ]:
# run the pipeline with default settings, and capture the outcome
info_1 = pipeline.run(
    people_1(),
    table_name='people',
    write_disposition='replace',
    primary_key='ID'
)

In [ ]:
info_2 = pipeline.run(
    people_2(),
    table_name='people',
    write_disposition='append',
    primary_key='ID'
)

In [ ]:
conn = duckdb.connect(f"{pipeline.pipeline_name}.duckdb")

In [ ]:
# Question 3
conn.sql("SELECT SUM(Age) FROM people.people")

In [ ]:
conn.close()

# 3. Merge a generator

Re-use the generators from Exercise 2.

A table's primary key needs to be created from the start, so load your data to a new table with primary key ID.

Load your first generator first, and then load the second one with merge. Since they have overlapping IDs, some of the records from the first load should be replaced by the ones from the second load.

After loading, you should have a total of 8 records, and ID 3 should have age 33.

Question: **Calculate the sum of ages of all the people loaded as described above.**


In [ ]:
# define the connection to load to
pipeline_merge = dlt.pipeline(destination='duckdb', dataset_name='people')

In [ ]:
# run the pipeline with default settings, and capture the outcome
merge_info_1 = pipeline_merge.run(
    people_1(),
    table_name='people_merged',
    write_disposition='replace',
    primary_key='ID'
)

In [ ]:
info_2 = pipeline.run(
    people_2(),
    table_name='people_merged',
    write_disposition='merge',
    primary_key='ID'
)

In [ ]:
conn = duckdb.connect(f"{pipeline_merge.pipeline_name}.duckdb")

In [ ]:
# Question 4
conn.sql("SELECT SUM(Age) FROM people.people_merged")

In [ ]:
conn.close()